<a href="https://colab.research.google.com/github/sriyanka-21/review_llm/blob/main/review_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install google-cloud-aiplatform
!pip install google-cloud-aiplatform transformers
!pip3 install --upgrade --user google-cloud-aiplatform
!pip install transformers
!pip install transformers pandas torch
!pip install pandas python-docx
!pip install nltk rouge-score transformers
!pip install pandas scikit-learn transformers textstat


In [55]:
from google.colab import auth
auth.authenticate_user()
from IPython.display import Markdown, display
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from textblob import TextBlob
import numpy as np
from google.cloud import aiplatform
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.language_models import (
    ChatModel,
    CodeChatModel,
    CodeGenerationModel,
    InputOutputTextPair,
    TextEmbeddingModel,
    TextGenerationModel,
)
from google.cloud import aiplatform

project_id = 'proven-episode-437804-q7'

aiplatform.init(project=project_id, location='us-central1')

In [ ]:

auth.authenticate_user()
project_id = 'proven-episode-437804-q7'
aiplatform.init(project=project_id, location='us-central1')


In [ ]:

product_df = pd.read_csv('product_asin.csv')
review_df = pd.read_csv('reviews_supplements.csv')


In [ ]:

print("Product Dataset Info:")
print(product_df.info())
print(product_df.head())

print("\nReview Dataset Info:")
print(review_df.info())
print(review_df.head())

print("\nRating Distribution:")
print(review_df['rating'].value_counts())


Product Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483354 entries, 0 to 483353
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   X            483354 non-null  int64 
 1   title        483329 non-null  object
 2   parent_asin  483354 non-null  object
 3   categories   483354 non-null  object
 4   cat1         483354 non-null  object
 5   cat2         465491 non-null  object
 6   cat3         449540 non-null  object
 7   cat4         296294 non-null  object
 8   cat5         96619 non-null   object
 9   cat6         22153 non-null   object
dtypes: int64(1), object(9)
memory usage: 36.9+ MB
None
    X                                              title parent_asin  \
0   1                     Allegra Allergy 45ct + 15 Free  B00JENH5OI   
1   2  InvoSpa Shiatsu Back Shoulder and Neck Massage...  B0C4L5Y711   
2   4  Kal 100 Mcg Selenium Yeast Free Tablets, 100 C...  B00020HX5S   
3   5  Rocky Mounta

In [ ]:

review_df.dropna(subset=['title', 'text'], inplace=True)

review_df['text'] = review_df['text'].str.lower()
review_df['text'] = review_df['text'].str.replace(r'[^\w\s]', '', regex=True)  # Remove punctuation


product_df.dropna(subset=['title'], inplace=True)

# Check for missing values after cleaning
print("\nMissing Values After Cleaning in Review Dataset:")
print(review_df.isnull().sum())
print("\nMissing Values After Cleaning in Product Dataset:")
print(product_df.isnull().sum())



Missing Values After Cleaning in Review Dataset:
rating               0
title                0
text                 0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
date                 0
time                 0
dtype: int64

Missing Values After Cleaning in Product Dataset:
X                   0
title               0
parent_asin         0
categories          0
cat1                0
cat2            17862
cat3            33811
cat4           187049
cat5           386715
cat6           461179
dtype: int64


In [ ]:

# Impute missing values in categorical columns for Product Dataset with the mode
for col in ['cat2', 'cat3', 'cat4', 'cat5', 'cat6']:
    mode_value = product_df[col].mode()[0]  # Get the mode for the column
    product_df[col] = product_df[col].fillna(mode_value)  # Replace NaN with mode

# For columns with very few missing values, consider using 'Unknown' as a placeholder
product_df['cat2'].fillna('Unknown', inplace=True)
product_df['cat3'].fillna('Unknown', inplace=True)

# Check if there are any missing values left in the Product Dataset
print("\nMissing Values After Imputation in Product Dataset:")
print(product_df.isnull().sum())

# Review Dataset
# For the review dataset, since title had some missing values, impute them with the mode
title_mode = review_df['title'].mode()[0]
review_df['title'].fillna(title_mode, inplace=True)

# Check if there are any missing values left in the Review Dataset
print("\nMissing Values After Imputation in Review Dataset:")
print(review_df.isnull().sum())


<ipython-input-21-6a8a33b1bea5>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  product_df['cat2'].fillna('Unknown', inplace=True)
<ipython-input-21-6a8a33b1bea5>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us


Missing Values After Imputation in Product Dataset:
X              0
title          0
parent_asin    0
categories     0
cat1           0
cat2           0
cat3           0
cat4           0
cat5           0
cat6           0
dtype: int64

Missing Values After Imputation in Review Dataset:
rating               0
title                0
text                 0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
date                 0
time                 0
dtype: int64


<ipython-input-21-6a8a33b1bea5>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  review_df['title'].fillna(title_mode, inplace=True)


In [ ]:
# Merging the Datasets
merged_df = pd.merge(review_df, product_df, on='parent_asin', how='inner')

# Rename the title columns to be more descriptive
merged_df.rename(columns={'title_x': 'review_title', 'title_y': 'product_title'}, inplace=True)


In [ ]:
Storing the Merged Dataset
merged_df.to_csv('merged_supplements_reviews.csv', index=False)
print("Merged dataset saved as 'merged_supplements_reviews.csv'.")

print("\nMerged Dataset:")
print(merged_df.head())


Merged dataset saved as 'merged_supplements_reviews.csv'.

Merged Dataset:
   rating                         review_title  \
0       4            B Complex in gel cap form   
1       5                           Five Stars   
2       5                           Five Stars   
3       5         Vitamin Shoppe Dry Vitamin A   
4       5  Un producto que compro regularmente   

                                                text        asin parent_asin  \
0  i bought this along with vit c in gel cap form...  B00012ND5G  B00012ND5G   
1                                      great product  B00013Z0ZQ  B00013Z0ZQ   
2                                 came as expectedly  B00013Z0ZQ  B00013Z0ZQ   
3  excellent product  fast delivery  will buy fro...  B00013Z1KA  B00013Z1KA   
4                              es muy buena vitamina  B00013Z1KA  B00013Z1KA   

                        user_id                timestamp  helpful_vote  \
0  AGDVFFLJWAQ3ULNNKF4LXID2RVSQ      2009-12-11 00:37:33             

In [ ]:

num_unique_products = review_df['parent_asin'].nunique()
print(f"Number of unique products: {num_unique_products}")

Number of unique products: 1498


In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
import random
import torch  # Ensure to import torch
import os

# Step 2: Load the merged dataset
merged_df = pd.read_csv('merged_supplements_reviews.csv')

# Initialize the BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Move the model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define output CSV file
output_file = 'synthetic_reviews.csv'

# Function to generate synthetic reviews in batches
def generate_reviews(product_title):
    # Define templates for positive, negative, and neutral reviews with short and long options
    templates = {
        'positive': {
            'short': [
                f"Fantastic {product_title}! I've noticed a significant improvement.",
                f"I'm impressed with {product_title}. It has exceeded my expectations.",
                f"Love {product_title}! Amazing results!",
                f"{product_title} is wonderful! Highly recommend it.",
                f"Great choice! {product_title} has been fantastic for me.",
                f"Absolutely fantastic product! {product_title} really works.",
                f"This is a must-have! {product_title} is so effective.",
                f"Highly recommend {product_title}! It really delivers.",
                f"Wonderful results with {product_title}. I'm very satisfied!",
                f"{product_title} is excellent! I would buy it again."
            ],
            'long': [
                f"This {product_title} is fantastic! I have noticed a significant improvement in my health since I started using it.",
                f"I'm really impressed with {product_title}. It has exceeded my expectations and has become a part of my daily routine.",
                f"Absolutely love {product_title}! I've experienced amazing results, and I can't recommend it enough.",
                f"{product_title} is a game changer! I feel more energetic and healthy ever since I started using it.",
                f"I highly recommend {product_title}. It has made a noticeable difference in my overall well-being.",
                f"I'm thrilled with {product_title}. It's effective, and I feel great using it!",
                f"This is the best {product_title} I've ever tried. Worth every penny!",
                f"{product_title} has really helped me out! I can't imagine my life without it now.",
                f"Such a great product! {product_title} has truly improved my health and vitality.",
                f"I'm so happy with {product_title}! It works wonders and tastes great too!"
            ]
        },
        'negative': {
            'short': [
                f"I'm not happy with {product_title}. It didn't work for me.",
                f"Disappointed with {product_title}. No noticeable effects.",
                f"I wouldn't recommend {product_title}. Waste of money.",
                f"{product_title} is not worth it. I was very disappointed.",
                f"Regrettably, {product_title} didn't meet my expectations.",
                f"{product_title} made me feel unwell. Not a fan.",
                f"I had high hopes for {product_title}, but it fell flat.",
                f"Don't waste your time on {product_title}. It's overrated.",
                f"Sadly, {product_title} did not work as advertised.",
                f"Honestly, I expected better from {product_title}."
            ],
            'long': [
                f"I'm not happy with {product_title}. It didn't meet my expectations and I felt no difference after using it.",
                f"Unfortunately, {product_title} did not work for me. I was really disappointed.",
                f"I had high hopes for {product_title}, but it fell flat. No noticeable effects.",
                f"{product_title} is overrated. I expected more based on the reviews but was let down.",
                f"I would not recommend {product_title}. It was a waste of money for me.",
                f"Regrettably, {product_title} didn't deliver the promised results.",
                f"I didn't like {product_title} at all. It made me feel unwell.",
                f"{product_title} was a letdown. I won't be purchasing it again.",
                f"Sadly, {product_title} did not work as advertised. Very disappointing.",
                f"Honestly, I expected better from {product_title}. I won't buy it again."
            ]
        },
        'neutral': {
            'short': [
                f"The {product_title} is okay. Some benefits but no major changes.",
                f"I think {product_title} is average. Does the job, but nothing extraordinary.",
                f"Decent product, {product_title} is just okay.",
                f"Nothing special about {product_title}. Just another supplement.",
                f"The {product_title} is decent. It hasn't changed my routine much.",
                f"Overall, {product_title} is just alright.",
                f"Neither good nor bad, {product_title} is acceptable.",
                f"{product_title} is okay, but I might try something else next time.",
                f"I'm indifferent about {product_title}. It's a regular product.",
                f"The {product_title} is fine, but not life-changing."
            ],
            'long': [
                f"The {product_title} is okay. It has some benefits, but I haven't noticed any major changes.",
                f"I think {product_title} is average. It does the job, but nothing extraordinary.",
                f"{product_title} works fine. I'm not overwhelmed, but it hasn't harmed me either.",
                f"Nothing special about {product_title}. It's just another supplement.",
                f"The {product_title} is decent. It hasn’t really changed my routine much.",
                f"Overall, {product_title} is just alright. I might consider trying something else next time.",
                f"The {product_title} is neither good nor bad. Just an okay addition to my diet.",
                f"I find {product_title} to be somewhat helpful, but not life-changing.",
                f"The {product_title} is what it is. It neither impresses nor disappoints.",
                f"I'm indifferent about {product_title}. It's simply another product in the market."
            ]
        }
    }

    generated_reviews = {
        'product_title': product_title,
        'reviews': []
    }

    # Generate one review for each sentiment
    for sentiment in templates.keys():
        review_length = random.choice(['short', 'long'])  # Randomly choose length for each review
        template = random.choice(templates[sentiment][review_length])  # Randomly select a template

        # Tokenize the input
        inputs = tokenizer.encode(template, return_tensors='pt', max_length=512, truncation=True).to(device)

        # Generate output from the model
        summary_ids = model.generate(inputs, max_length=100, num_beams=4, length_penalty=2.0, early_stopping=True)
        generated_review = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        generated_reviews['reviews'].append({
            'sentiment': sentiment,
            'review': generated_review
        })

        # Print the generated review
        print(f"Generated {sentiment.capitalize()} Review for {product_title}: {generated_review}")

        # Save the review to CSV one by one
        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(f"{product_title},{sentiment},{generated_review}\n")

    return generated_reviews

# Step 4: Iterate over each unique product and generate reviews
# Create CSV file and write header
if not os.path.exists(output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("product_title,sentiment,review\n")

# Generate reviews for each unique product title
unique_product_titles = merged_df['product_title'].unique()

for product_title in unique_product_titles:
    generate_reviews(product_title)

print("All synthetic reviews have been saved incrementally in 'synthetic_reviews.csv'.")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Generated Positive Review for BComplex 50 – Supports Energy Production, Nervous System Function Nutrient Metabolism – Excellent Source of B1, B2, B6, B12, Niacin, Folic Acid Biotin (100 Softgels) by The Vitamin Shoppe: Absolutely fantastic product! BComplex 50 – Supports Energy Production, Nervous System Function Nutrient Metabolism – Excellent Source of B1, B2, B6, B12, Niacin, Folic Acid Biotin (100 Softgels) by The Vitamin Shoppe really works.
Generated Negative Review for BComplex 50 – Supports Energy Production, Nervous System Function Nutrient Metabolism – Excellent Source of B1, B2, B6, B12, Niacin, Folic Acid Biotin (100 Softgels) by The Vitamin Shoppe: I wouldn't recommend BComplex 50 – Supports Energy Production, Nervous System Function Nutrient Metabolism – Excellent Source of B1, B2, B6, B12, Niacin, Folic Acid Biotin (100 Softgels) by The Vitamin Shoppe. Waste of money.
Generated Neutral Review for BComplex 50 – Supports Energy Production, Nervous System Function Nutrient 

KeyboardInterrupt: 

In [46]:
import re
import pandas as pd
from docx import Document  # Import the Document class from python-docx

# Define the input and output files
input_file = 'synthetic.docx'
output_file = 'structured_reviews.csv'

# Create a list to store the extracted reviews
reviews_data = []

# Read the input .docx file
doc = Document(input_file)
for para in doc.paragraphs:
    line = para.text.strip()  # Get the paragraph text
    # Use regex to extract the sentiment, product title, and review
    match = re.match(r"Generated (\w+) Review for (.*?): (.*)", line)
    if match:
        sentiment = match.group(1)  # Sentiment (Positive, Negative, Neutral)
        product_title = match.group(2).strip()  # Product title
        review = match.group(3).strip()  # The actual review text

        # Append to the list
        reviews_data.append({
            'product_title': product_title,
            'sentiment': sentiment,
            'review': review
        })

# Convert the list to a DataFrame
reviews_df = pd.DataFrame(reviews_data)

# Save to CSV
reviews_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"Structured reviews have been saved to '{output_file}'.")


Structured reviews have been saved to 'structured_reviews.csv'.


In [52]:
import pandas as pd
from textstat import flesch_reading_ease, flesch_kincaid_grade

# Load the structured reviews dataset
reviews_df = pd.read_csv('structured_reviews.csv')

# Descriptive Statistics
print("Descriptive Statistics:")
reviews_df['review_length'] = reviews_df['review'].apply(len)
print(f"Average Review Length: {reviews_df['review_length'].mean():.2f}")
print(f"Median Review Length: {reviews_df['review_length'].median():.2f}")
print(f"Sentiment Distribution:\n{reviews_df['sentiment'].value_counts()}\n")

# Readability Metrics
reviews_df['flesch_reading_ease'] = reviews_df['review'].apply(flesch_reading_ease)
reviews_df['flesch_kincaid_grade'] = reviews_df['review'].apply(flesch_kincaid_grade)

print("Readability Metrics:")
print(f"Average Flesch Reading Ease: {reviews_df['flesch_reading_ease'].mean():.2f}")
print(f"Average Flesch-Kincaid Grade Level: {reviews_df['flesch_kincaid_grade'].mean():.2f}\n")

# Review Length Statistics
print("Review Length Statistics:")
print(reviews_df['review_length'].describe())
print(f"Unique Reviews Count: {len(reviews_df)}")


Descriptive Statistics:
Average Review Length: 124.21
Median Review Length: 114.00
Sentiment Distribution:
sentiment
Positive    453
Negative    453
Neutral     452
Name: count, dtype: int64

Readability Metrics:
Average Flesch Reading Ease: 60.90
Average Flesch-Kincaid Grade Level: 7.56

Review Length Statistics:
count    1358.000000
mean      124.208395
std        48.180957
min        43.000000
25%        93.000000
50%       114.000000
75%       142.750000
max       420.000000
Name: review_length, dtype: float64
Unique Reviews Count: 1358
